In [ ]:
# 解析
# データの取得
!wget http://misc.0093.tv/misc/kadai.xlsx

In [ ]:
!pip install optuna

In [ ]:
# ライブラリのインポート
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# データの確認
df = pd.read_excel("kadai.xlsx")
df.head(5)

In [ ]:
# 基本統計量
df.describe().T[['count','mean','std','min','50%','max']]

In [ ]:
# 相関行列の確認（多重共線性のチェック）
plt.figure(figsize=(10, 8))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=False, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# 変数の指定
target = 'OV'
features = df.drop(columns=[target]).columns.tolist()

In [ ]:
# 前処理と外れ値の処理
# 0. "process_end_time"と"final_mes_time"を時系列オブジェクトに変換する
df["process_end_time"] = pd.to_datetime(df["process_end_time"])
df["final_mes_time"] = pd.to_datetime(df["final_mes_time"])

# 1. 欠損値の確認
df.info()

# 2. 外れ値の確認(IQR法)
Q1 = df[target].quantile(0.25)
Q3 = df[target].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliners = df[(df[target] < lower_bound) | (df[target] > upper_bound)]
print(f"目的変数{target}の外れ値：{len(outliners)}個")
print(outliners[[target]])

In [ ]:
# 特徴量エンジニアリング
# 1. Lag特徴量
lags = [1, 2, 3, 5]
for lag in lags:
  df[f"{target}_lag{lag}"] = df[target].shift(lag)

# 2. 差分特徴量
df[f"{target}_diff"] = df[target].diff(1).shift(1)

# 3. 移動平均と移動標準偏差
windows = [3, 5]
for window in windows:
  df[f"{target}_roll_mean{window}"] = df[target].rolling(window).mean().shift(1)
  df[f"{target}_roll_std{window}"] = df[target].rolling(window).std().shift(1)

# 4. Shiftへの対応
df = df.dropna().reset_index(drop=True)

In [ ]:
# 変数選択

In [ ]:
# モデルのパラメータ(線形回帰、勾配ブースティング回帰、RF、XGB、SVM、LightGBM)
tscv = TimeSeriesSplit(n_splits=5)

# 線形回帰
def objective_linear(trial):
    param = {
        'alpha': trial.suggest_categorical('alpha', [0.1, 1, 10]),
        'max_iter': 100000,
        'random_state': 42
    }

    model = Lasso(**param)

    scores = cross_val_score(model, X_init_origin, y_init_origin, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# 勾配ブースティング回帰
def objective_gbr(trial):
    param = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 300, 500]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7]),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 2, 4]),
        'subsample': trial.suggest_categorical('subsample', [0.8, 1.0]),
        'loss': trial.suggest_categorical('loss', ['squared_error', 'huber']),
        'random_state': 42
    }

    model = GradientBoostingRegressor(**param)

    scores = cross_val_score(model, X_init_scaled, y_init, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# ランダムフォレスト
def objective_rf(trial):
    param = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 300, 500]),
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7, 9]),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5]),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 2, 4, 10]),
        'random_state': 42,
        'n_jobs': -1
    }

    model = RandomForestRegressor(**param)
    scores = cross_val_score(model, X_init_scaled, y_init, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# XGBoost
def objective_xgb(trial):
    param = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.05, 0.1, 0.2, 0.3]),
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7]),
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': trial.suggest_categorical('reg_alpha', [0, 0.1, 1, 10]),
        'reg_lambda': trial.suggest_categorical('reg_lambda', [1, 5, 9]),
        'n_estimators': 1000,
        'random_state': 42,
        'n_jobs': -1
    }

    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, X_init_scaled, y_init, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# SVM
def objective_svm(trial):
    param = {
        'C': trial.suggest_categorical('C', [0.1, 1, 10, 100]),
        'epsilon': trial.suggest_categorical('epsilon', [0.01, 0.1, 0.5]),
        'kernel': 'rbf',
        'gamma': trial.suggest_categorical('gamma', ['scale', 0.001, 0.01, 0.1])
    }

    model = SVR(**param)
    scores = cross_val_score(model, X_init_scaled, y_init, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# LightGBM
def objective_lgb(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'min_child_samples': 20,
        'n_estimators': 1000,
        'random_state': 42,
        'verbose': -1
    }

    model = lgb.LGBMRegressor(**param)

    scores = cross_val_score(model, X_init_scaled, y_init, cv=tscv, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [ ]:
# 最適モデルの保存場所
from google.colab import drive
import os
import joblib

# ドライブをマウント（初回のみ認証が必要）
drive.mount('/content/drive')

# 保存用ディレクトリを作成（エラーにならないよう exist_ok=True）
save_dir = '/content/drive/MyDrive/Colab_ML_Params'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
models = {}

In [ ]:
# パラメータチューニング
file_path = save_dir

# パラメータチューニング用データ
learn_init = df[0:1776]
#X_init = learn_init.drop(columns=['process_end_time','final_mes_time',target], axis=1)
y_init = learn_init[target]
X_init = learn_init[selected_features]

learn_init1 = df1[0:1776]
#X_init_origin = learn_init1.drop(columns=['process_end_time','final_mes_time',target], axis=1)
y_init_origin = learn_init1[target]
X_init_origin = learn_init1[selected_features]

# 標準化(SVM用)
scaler = StandardScaler()
X_init_scaled = scaler.fit_transform(X_init)
scaler2 = StandardScaler()
X_init_origin = scaler2.fit_transform(X_init_origin)

# 線形回帰
print("---------- 線形回帰モデル(Lasso) ----------")
lr_init = optuna.create_study(direction='minimize')
lr_init.optimize(objective_linear, n_trials=10)
lr_best_params = lr_init.best_params
best_lr = Lasso(**lr_best_params)
print(f"線形回帰　採用されたパラメータ：{lr_best_params}")
models['Lasso'] = best_lr
print("-------------------------------------------")
joblib.dump(lr_best_params, os.path.join(file_path, 'lr_best_params.pkl'))

# 勾配ブースティング回帰
print("---------- 勾配ブースティング回帰 ----------")
gbr_init = optuna.create_study(direction='minimize')
gbr_init.optimize(objective_gbr, n_trials=20)
gbr_best_params = gbr_init.best_params
best_gbr = GradientBoostingRegressor(**gbr_best_params)
print(f"勾配ブースティング回帰　採用されたパラメータ{gbr_best_params}")
models['GBR'] = best_gbr
print("-----------------------------------------------------------")
joblib.dump(gbr_best_params, os.path.join(file_path, 'gbr_best_params.pkl'))

# ランダムフォレスト
print("---------- ランダムフォレスト(RandomForest) ----------")
rf_init = optuna.create_study(direction='minimize')
rf_init.optimize(objective_rf, n_trials=20)
rf_best_params = rf_init.best_params
best_rf = RandomForestRegressor(**rf_best_params)
print(f"ランダムフォレスト　採用されたパラメータ{rf_best_params}")
models['RF'] = best_rf
print("------------------------------------------------------------")
joblib.dump(rf_best_params, os.path.join(file_path, 'rf_best_params.pkl'))

# XGBoost
print("---------- XGBoost(eXtreme Gradient Boosting) ----------")
xgb_init = optuna.create_study(direction='minimize')
xgb_init.optimize(objective_xgb, n_trials=20)
xgb_best_params = xgb_init.best_params
best_xgb = xgb.XGBRegressor(**xgb_best_params)
print(f"XGBoost　採用されたパラメータ{xgb_best_params}")
models['XGB'] = best_xgb
print("--------------------------------------------------------")
joblib.dump(xgb_best_params, os.path.join(file_path, 'xgb_best_params.pkl'))

# SVM
print("---------- SVM(Support Vector Machine) ----------")
svm_init = optuna.create_study(direction='minimize')
svm_init.optimize(objective_svm, n_trials=20)
svm_best_params = svm_init.best_params
best_svm = SVR(**svm_best_params)
print(f"SVM　採用されたパラメータ{svm_best_params}")
models['SVM'] = best_svm
print("-------------------------------------------------")
joblib.dump(svm_best_params, os.path.join(file_path, 'svm_best_params.pkl'))

# LightGBM
print("---------- LightGBM ----------")
gbm_init = optuna.create_study(direction='minimize')
gbm_init.optimize(objective_lgb, n_trials=10)
gbm_best_params = gbm_init.best_params
best_gbm = lgb.LGBMRegressor(**gbm_best_params)
print(f"LightGBM　採用されたパラメータ{gbm_best_params}")
models['LightGBM'] = best_gbm
print("-----------------------------")
joblib.dump(gbm_best_params, os.path.join(file_path, 'gbm_best_params.pkl'))

In [ ]:
# 最適モデルのRMSE
print("BEST RMSE")
print(f"線形回帰モデル　{lr_init.best_value}")
print(f"勾配ブースティング回帰　{gbr_init.best_value}")
print(f"RandomForest　{rf_init.best_value}")
print(f"XGBoost　{xgb_init.best_value}")
print(f"SVM　{svm_init.best_value}")
print(f"LightGBM　{gbm_init.best_value}")

In [ ]:
y_Hat = []
y_svm = []
y_lr = []

end = min(2276, len(df))

for i in np.arange(1776, end):
    if (i - 1776) % 50 == 0:
        print(f"Processing index: {i} / 2276")

    learn = df[0:i].copy().dropna()
    test = df[i:i+1].copy()

    learn = learn.reset_index(drop=True)
    test = test.reset_index(drop=True)

    learn = learn[learn["final_mes_time"] < test['process_end_time'][0]]

    X_l = learn.drop(columns=['process_end_time','final_mes_time',target], axis=1)
    y_l = learn[target]
    X_t = test.drop(columns=['process_end_time','final_mes_time',target], axis=1)

    scaler_x = StandardScaler()
    X_l_scaled = scaler_x.fit_transform(X_l)
    X_t_scaled = scaler_x.transform(X_t)

    best_svm.fit(X_l_scaled, y_l)
    best_lr.fit(X_l_scaled, y_l)

    pred_train_svm = best_svm.predict(X_l_scaled)
    pred_train_lr = best_lr.predict(X_l_scaled)
    
    base_train_pred = pred_train_svm * 0.6 + pred_train_lr * 0.4
    residuals = y_l - base_train_pred

    best_rf.fit(X_l_scaled, residuals)

    pred_svm = best_svm.predict(X_t_scaled)
    pred_lr = best_lr.predict(X_t_scaled)
    
    pred_resid = best_rf.predict(X_t_scaled)

    y_Hat.append((pred_svm[0] * 0.6 + pred_lr[0] * 0.4 + pred_resid[0]))
    y_svm.append(pred_svm[0])
    y_lr.append(pred_lr[0])

Y_t = df["OV"][1776:2276].reset_index(drop=True)
Y_t = Y_t.reset_index()['OV']

plt.figure(figsize=(12, 6))
plt.plot(Y_t, label='Actual OV', color='black', linestyle='--')

yh = np.array(y_Hat)
rmse = np.sqrt(mean_squared_error(Y_t, yh))
plt.plot(yh, label=f'Ensemble+Resid RMSE: {rmse:.2f}', alpha=0.8)
print(f"アンサンブル学習(残差補正あり): {rmse}")

yh_svm = np.array(y_svm)
rmse_svm = np.sqrt(mean_squared_error(Y_t, yh_svm))
plt.plot(yh_svm, label=f'SVM RMSE: {rmse_svm:.2f}', alpha=0.8)
print(f"SVM: {rmse_svm}")

yh_lr = np.array(y_lr)
rmse_lr = np.sqrt(mean_squared_error(Y_t, yh_lr))
plt.plot(yh_lr, label=f'Linear Regression RMSE: {rmse_lr:.2f}', alpha=0.8)
print(f"線形回帰: {rmse_lr}")

plt.legend()
plt.title("Prediction Comparison with Residual Learning")
plt.show()